In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import re
import random
from sklearn.model_selection import train_test_split
import pickle
import tensorflow as tf
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.callbacks import ModelCheckpoint
import time
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
from numpy import dot
from numpy.linalg import norm
import pickle
from tensorflow.keras.utils import to_categorical  ## maintain tf.keras always

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [3]:
RANDOM_STATE = 2245 # for ensuring reproducible results.
tf.random.set_seed(RANDOM_STATE)
np.random.seed(RANDOM_STATE)

In [4]:
def cosine_similarity(a, b):
### https://stackoverflow.com/questions/18424228/cosine-similarity-between-2-number-lists
    cos_sim = np.inner(a, b) / (norm(a) * norm(b))    
    return cos_sim

def get_various_metrics_and_print(Y_true, Y_predicted):
    TN, FP, FN, TP = confusion_matrix(Y_true, Y_predicted).ravel()
    accuracy = (TP + TN)/(TP+TN+FP+FN)
    recall = (TP)/(TP + FN)
    specificity = (TN)/(TN + FP) # TNR
    false_positive_rate = (FP)/(TN + FP) # false_positive_rate = 1 - TNR
    precision = (TP)/(TP + FP)
    false_discovery_rate = (FP)/(TP + FP)
    neg_predicted_val = (TN)/(TN + FN)
    f1_score = 2*((precision * recall) / (precision + recall))

    print("TN = ", TN, " FP = ", FP, " FN = ", FN, " TP = ", TP)
    print("Accuracy = ", accuracy*100, "%")
    print("TPR = Sensitivity = Recall = ", recall*100, "%")
    print("TNR = Specificity = ", specificity*100, "%")
    print("Precision = PPV = Positive Predictive Value = ", precision*100, "%")
    print("FDR = False Discovery Rate = ", false_discovery_rate*100, "%")
    print("FPR = False Positive Rate = ", false_positive_rate*100, "%")
    print("F1 Score = ", f1_score*100, "%")
    print("Neg Predicted Val = ", neg_predicted_val*100, "%")
    print("\n")
    c_report = classification_report(y_true=Y_true, y_pred=Y_predicted)
    print(c_report)
    # return precision, recall, f1_score

def plot_confusion_matrix(Y_true=None, Y_predicted=None):
    ## https://medium.com/@dtuk81/confusion-matrix-visualization-fc31e3f30fea
    cf_matrix = confusion_matrix(Y_true, Y_predicted)
    print(cf_matrix)
    # group_names = ['True Neg','False Pos','False Neg','True Pos']
    group_names = ['TN','FP','FN','TP']
    group_counts = ["{0:0.0f}".format(value) for value in cf_matrix.flatten()]
    group_percentages = ["{0:.2%}".format(value) for value in cf_matrix.flatten()/np.sum(cf_matrix)]
    labels = [f"{v1}\n{v2}\n{v3}" for v1, v2, v3 in zip(group_names,group_counts,group_percentages)]
    labels = np.asarray(labels).reshape(2,2)
    sns.heatmap(cf_matrix, annot=labels, fmt='', cmap=plt.cm.magma)
    plt.xlabel("Predicted")
    plt.ylabel("Actual")

# Load the fake-news data and do pre-processing

In [5]:
DATE_TIME = "22-Aug-2am"
NAME = "bi-LSTM2"
FILE_TRAIN_AND_VALIDATION = "/content/drive/My Drive/Data_Initial/contents-labels-train-val.csv"
file_stop_words_bangla = "/content/drive/My Drive/Data_Initial/STOP_WORDS_BANGLA.txt"
FILE_MODEL = "/content/drive/My Drive/SAVED_MODELS/classifier-" + NAME + "-" + DATE_TIME + ".h5"
#FILE_MODEL = "/content/drive/My Drive/SAVED_MODELS/emb-content_with_synthsis" + DIMENSIONS + "-" + DATE_TIME + ".h5"

FILE_PATH_WEIGHTS_MODEL="/content/drive/My Drive/SAVED_MODELS/Weights/classifier-" + NAME + "-" + DATE_TIME + "-weights.best.hdf5"

print(f"FILE_MODEL = {FILE_MODEL}")

FILE_MODEL = /content/drive/My Drive/SAVED_MODELS/classifier-bi-LSTM2-22-Aug-2am.h5


In [6]:
df_whole = pd.read_csv(FILE_TRAIN_AND_VALIDATION)
df_whole = df_whole.sample(frac=1, random_state=RANDOM_STATE)

In [7]:
"""
  Loads the stop words into a list.
"""
def load_stop_words_list_and_dict(file):
  fin = open(file, 'r', encoding="utf-8") # open file
  st_words = fin.readlines() # load st-words into variable [list]
  fin.close() # close file
  stop_words_list = []
  for word in st_words: # remove new-lines/other spaces
    word = word.strip()
    stop_words_list.append(word)
  
  stop_words_dict = {} # dictionary of key-value pairs. [to do easy checking]
  for sw in stop_words_list:
          stop_words_dict[sw.strip()] = "" # for each stop word, it's replacement will be an empty-string
          
  return stop_words_list, stop_words_dict

stop_words_list, stop_words_dict = load_stop_words_list_and_dict(file_stop_words_bangla)
print(len(stop_words_list))
print(stop_words_list[0:10])
print(stop_words_dict)

430
['অতএব', 'অথচ', 'অথবা', 'অনুযায়ী', 'অনেক', 'অনেকে', 'অনেকেই', 'অন্তত', 'অন্য', 'অবধি']
{'অতএব': '', 'অথচ': '', 'অথবা': '', 'অনুযায়ী': '', 'অনেক': '', 'অনেকে': '', 'অনেকেই': '', 'অন্তত': '', 'অন্য': '', 'অবধি': '', 'অবশ্য': '', 'অর্থাত': '', 'আই': '', 'আগামী': '', 'আগে': '', 'আগেই': '', 'আছে': '', 'আজ': '', 'আদ্যভাগে': '', 'আপনার': '', 'আপনি': '', 'আবার': '', 'আমরা': '', 'আমাকে': '', 'আমাদের': '', 'আমার': '', 'আমি': '', 'আর': '', 'আরও': '', 'আরো': '', 'ই': '', 'ইত্যাদি': '', 'ইহা': '', 'উচিত': '', 'উত্তর': '', 'উনি': '', 'উপর': '', 'উপরে': '', 'এ': '', 'এঁদের': '', 'এঁরা': '', 'এই': '', 'একজন': '', 'এক': '', 'একটা': '', 'একই': '', 'একটি': '', 'একবার': '', 'একে': '', 'এক্': '', 'এখন': '', 'এখনও': '', 'এখানে': '', 'এখানেই': '', 'এটা': '', 'এটাই': '', 'এটি': '', 'এত': '', 'এতটাই': '', 'এতে': '', 'এদের': '', 'এব': '', 'এবং': '', 'এবার': '', 'এমন': '', 'এমনকী': '', 'এমনি': '', 'এর': '', 'এরপর': '', 'এরা': '', 'এল': '', 'এস': '', 'এসে': '', 'ঐ': '', 'ও': '', 'ওঁদের': '', 'ওঁর': '', 'ওঁর

In [8]:
"""
  Given a sentence, it removes the stop-words and other things [stemming]
  Input: sentence, list of stop-words
  Output: stemmed sentence
"""
def obtain_stemmed_sentence(sentence, stop_words_dict): # Base code: Rafi
  stemmed_sentence = sentence # first copy the sentence
  stemmed_sentence = stemmed_sentence.replace("\n", "") # replace new line with empty-string
  # remove english words
  stemmed_sentence = re.sub(r'[।\\/“”0-9@&$%+_=<>~*#০১২৩৪৫৬৭৮৯…(){}\[\]\?\!\.a-zA-Z\…\|]+','',stemmed_sentence) # replace theese with empty-string [Mahim]
  sentences_split = re.split('[–—\s.,\-\‘\’\‘\'\\\":]',stemmed_sentence) # split wrt these delims
  label = ""
  for x in sentences_split:
    if x not in stop_words_dict and len(x) >= 2: # if word is not a stop-word append by adding a space
        label = label + " " + x
#   label = label.replace("  ", " ") # replace double spaces with one space
  return label

In [9]:
# ################## Obtain the dictionary of vocab ##################
# list_contents_whole = df_whole["content"]
# dict_vocab = {}
# for content in list_contents_whole:
#     stemmed_content = obtain_stemmed_sentence(content, stop_words_dict)
#     ### list_document_level_content_stemmed.append(stemmed_content)
#     words = stemmed_content.split(" ")
#     for word in words:
#         if word not in dict_vocab and len(word)>1:
#             dict_vocab[word] = 0

In [10]:
"""
    Utility function to print first n keys of a dictionary along with its values
"""
def print_first_n_keys(dictionary_, n=10):
    idx = 0
    for key in dictionary_:
        idx += 1
        if idx > n:
            break
        print(key, dictionary_[key])

# print(len(dict_vocab))
# print_first_n_keys(dict_vocab, 10)

In [11]:
# write to 
# print_first_n_keys(dict_vocab, len(dict_vocab)) # there exists an english word [HOT]
def write_to_file_vocab(): # Expensive, use once.
    out_file = "/content/drive/My Drive/Data_Initial/Bangla-Vocab-22-Aug-withoutEnglish.txt"
    print(f"Writing to {out_file}")
    f = open(out_file, "w")
    # f.write("Now the file has more content!")
    for key in dict_vocab:
        f.write(key)
        f.write("\n")
    f.close()

def read_from_file_and_load_vocab():
    vocab_list = []
    in_file = "/content/drive/My Drive/Data_Initial/Bangla-Vocab-22-Aug-withoutEnglish.txt"
    print(f"in_file = {in_file}")
    fin1 = open(in_file,'r', encoding="utf-8")
    for line in fin1.readlines():
        if len(line) > 1:
            vocab_list.append(line.strip())
    fin1.close()
    return vocab_list


## write_to_file_vocab()
vocab_list = read_from_file_and_load_vocab()
print("Size of list of vocab", len(vocab_list))

in_file = /content/drive/My Drive/Data_Initial/Bangla-Vocab-22-Aug-withoutEnglish.txt
Size of list of vocab 237931


In [12]:
def get_vocab_dictionary(vocab_list):
    vocab_dict = {}
    cnt = 0
    for word in vocab_list:
        if word not in vocab_dict:
            vocab_dict[word] = cnt
        cnt += 1
    return vocab_dict

vocab_dict = get_vocab_dictionary(vocab_list)
print(len(vocab_dict))

237931


In [13]:
vocab_size = len(vocab_list) + 1
print(vocab_size)
del vocab_list
# encoded_contents = [one_hot(d, vocab_size) for d in list_document_level_content_stemmed]

237932


In [14]:
def get_encoded_content(stemmed_sentence, vocab_dict):
    list_encoded_content = []
    for word in stemmed_sentence.split():
        if word in vocab_dict:
            encoded_int = vocab_dict[word] + 1 # since '0' is used for padding
            list_encoded_content.append(encoded_int)
    return list_encoded_content

In [15]:
LEN_DOC_CONTENT = 400 # Aug 9 and above

In [16]:
df_whole["stemmed_content"] = df_whole.apply(lambda row : obtain_stemmed_sentence(row['content'], stop_words_dict), axis=1)

**Only Run the following cell for synthetic_samples**

In [ ]:

stemmed_docs=df_whole["stemmed_content"].values
labels=df_whole["label"].values
df_whole=df_whole.drop(columns=["stemmed_content","label"])

file=open('/content/drive/My Drive/fake-news/generated_stemmed_fake.pkl','rb')
synth_fake=pickle.load(file)
synth_fake.shape

stemmed_docs=np.concatenate([stemmed_docs,synth_fake])
stemmed_docs.shape
labels=np.concatenate([labels,np.zeros(len(synth_fake))])
labels.shape

df_new= pd.DataFrame(list(zip(stemmed_docs, labels)), 
               columns =['stemmed_content', 'label']) 
df_whole=df_new
df_whole=df_whole.sample(random_state=RANDOM_STATE,frac=1)
df_whole.head
FILE_MODEL = "/content/drive/My Drive/SAVED_MODELS/classifier_with_synthsis-" + NAME + "-" + DATE_TIME + ".h5"
FILE_PATH_WEIGHTS_MODEL="/content/drive/My Drive/SAVED_MODELS/Weights/classifier_with_synthsis-" + NAME + "-" + DATE_TIME + "-weights.best.hdf5"


In [17]:
df_whole["encoded_contents"] = df_whole.apply(lambda row : get_encoded_content(row['stemmed_content'], vocab_dict=vocab_dict), axis=1)

In [18]:
print(df_whole.iloc[0]["stemmed_content"])
print(df_whole.iloc[0]["encoded_contents"])

 নিউজ ডেস্ক আরটিএনএন ঢাকা বিদেশে মনগড়া বই লিখেছেন সাবেক প্রধান বিচারপতি সিনহা মন্তব্য আওয়ামী লীগের সম্পাদক যোগাযোগমন্ত্রী ওবায়দুল কাদের বাংলাদেশের সাবেক প্রধান বিচারপতি সুরেন্দ্র কুমার সিনহা আত্মজীবনীমূলক বই প্রকাশ দাবি সরকার সামরিক গোয়েন্দা সংস্থার চাপ হুমকির মুখে দেশত্যাগ বিচারপতি সিনহার বই ব্রোকেন ড্রিম রুল অব হিউম্যান রাইটস অ্যান্ড ডেমোক্রেসি মাত্রই প্রকাশিত বৃহস্পতিবার ঢাকায় বঙ্গবন্ধু অ্যাভিনিউতে আওয়ামী লীগের কেন্দ্রীয় কার্যালয়ে অনুষ্ঠানে বইটি সাংবাদিকরা প্রশ্ন ওবায়দুল কাদের জবাবে ক্ষমতা হারানোর জ্বালা বিচারপতি সিনহা বই লিখে মনগড়া বলছেন ক্ষমতায় অন্তরজ্বালা বেদনা অন্তরজ্বালা বিদেশে এরকম বলার প্রয়োজন আওয়ামী লীগের সম্পাদক প্রধান বিচারপতি অবস্থায় বলছেন বইতে বলার সাহস বিচারপতির নৈতিকতার প্রশ্ন বলছেন কাদের বইয়ে বিচারপতি সিনহা সবিস্তারে বর্ণনা পরিপ্রেক্ষিতে সরকারের বিরোধ তৈরি বইয়ে দাবি কিভাবে দেশত্যাগে বাধ্য প্রধান বিচারপতির পদ পদত্যাগে বাধ্য দাবি বাংলাদেশের সামরিক গোয়েন্দা সংস্থা ডিজিএফআইয়ের হুমকি ভীতি প্রদর্শনের মুখে দেশে ছেড়েছেন বিচারপতি সিনহা লিখেছেন বাংলাদেশের সংব

## Remove empty or one-word ENCODED contents

In [19]:
## https://thispointer.com/python-pandas-how-to-drop-rows-in-dataframe-by-conditions-on-column-values/
## https://stackoverflow.com/questions/19937362/python-pandas-filter-string-data-based-on-its-string-length

MIN_WORD_COUNT_TO_REMOVE = 3
print(df_whole[df_whole['encoded_contents'].str.len() <= MIN_WORD_COUNT_TO_REMOVE]["stemmed_content"])
idxToRemove = df_whole[df_whole['encoded_contents'].str.len() <= MIN_WORD_COUNT_TO_REMOVE].index
df_whole.drop(idxToRemove , inplace=True)

27283     নমুনা
28619          
30510          
12633          
45047          
7609           
19577          
Name: stemmed_content, dtype: object


In [20]:
"""
lables = df_whole["label"].values

pkl_file = open('drive/My Drive/fake-news/train_labels.pkl', 'wb')
pickle.dump(stemmed_contents,pkl_file)
pkl_file.close()
"""

'\nlables = df_whole["label"].values\n\npkl_file = open(\'drive/My Drive/fake-news/train_labels.pkl\', \'wb\')\npickle.dump(stemmed_contents,pkl_file)\npkl_file.close()\n'

In [21]:
# df_whole["padded_encoded_contents"] = df_whole.apply(lambda row : pad_encoded_content(row['encoded_contents'], len_to_pad=LEN_DOC_CONTENT), axis=1)
arr = np.array([len(x) for x in df_whole["encoded_contents"].values])
arr.sort()
print(arr[0:10])
print(df_whole.columns.values)

[4 4 5 5 5 5 5 5 5 5]
['content' 'label' 'stemmed_content' 'encoded_contents']


In [22]:
idx=0
list_indices = []
for x in df_whole['stemmed_content'].values:
    if len(x) <= 6:
        # print(idx, x)
        list_indices.append(idx)
    idx+= 1

print(list_indices)

[]


In [23]:
## to get f1-score while training ##
## https://medium.com/@aakashgoel12/how-to-add-user-defined-function-get-f1-score-in-keras-metrics-3013f979ce0d
from sklearn.utils import shuffle
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
import tensorflow.keras.backend as K

In [24]:
def get_f1(y_true, y_pred): #taken from old keras source code
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val

def get_recall(y_true, y_pred): #taken from old keras source code
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return recall

## LSTM Model.

In [ ]:
emb_dim=32 # need this
emb_model_content = tf.keras.Sequential([ 
    # tf.keras.layers.Input(shape=(MAX_LEN_DOC_CONTENT,)) , 
    tf.keras.layers.Input(shape=(LEN_DOC_CONTENT,)) , 
    tf.keras.layers.Embedding(vocab_size, output_dim=emb_dim),
    tf.keras.layers.LSTM(128,return_sequences=True,dropout=0.3),
    tf.keras.layers.LSTM(64,return_sequences=True,dropout=0.2),
    tf.keras.layers.LSTM(32),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(8, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    # tf.keras.layers.Dense(4, activation='relu'),
    # tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(2, activation="softmax")
    # tf.keras.layers.Dense(1, activation="sigmoid")
])

emb_model_content.load_weights(FILE_PATH_WEIGHTS_MODEL)

#emb_model_content.load_weights("/content/drive/My Drive/SAVED_MODELS/Weights/emb-content_with_synthsis-32D-22-Aug-2am-weights.best.hdf5")
emb_model_content.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc', get_f1, get_recall])
print(emb_model_content.summary())

# Creating best-model checkpoint

checkpoint = ModelCheckpoint(FILE_PATH_WEIGHTS_MODEL, monitor='val_get_f1', verbose=1, save_best_only=True, mode='max')
# checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

print(f"FILE_PATH_WEIGHTS_EMB_MODEL = {FILE_PATH_WEIGHTS_MODEL}")

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 400, 32)           7613824   
_________________________________________________________________
lstm_3 (LSTM)                (None, 400, 128)          82432     
_________________________________________________________________
lstm_4 (LSTM)                (None, 400, 64)           49408     
_________________________________________________________________
lstm_5 (LSTM)                (None, 32)                12416     
_________________________________________________________________
dense_3 (Dense)              (None, 16)                528       
_________________________________________________________________
dropout_2 (Dropout)          (None, 16)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 8)                

## GRU Model.

In [ ]:
emb_dim=32 # need this
emb_model_content = tf.keras.Sequential([ 
    # tf.keras.layers.Input(shape=(MAX_LEN_DOC_CONTENT,)) , 
    tf.keras.layers.Input(shape=(LEN_DOC_CONTENT,)) , 
    tf.keras.layers.Embedding(vocab_size, output_dim=emb_dim),
    tf.keras.layers.GRU(128,return_sequences=True,dropout=0.3),
    tf.keras.layers.GRU(64,return_sequences=True,dropout=0.2),
    tf.keras.layers.GRU(32),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(8, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    # tf.keras.layers.Dense(4, activation='relu'),
    # tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(2, activation="softmax")
    # tf.keras.layers.Dense(1, activation="sigmoid")
])

emb_model_content.load_weights(FILE_PATH_WEIGHTS_MODEL)

#emb_model_content.load_weights("/content/drive/My Drive/SAVED_MODELS/Weights/emb-content_with_synthsis-32D-22-Aug-2am-weights.best.hdf5")
emb_model_content.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc', get_f1, get_recall])
print(emb_model_content.summary())

# Creating best-model checkpoint

checkpoint = ModelCheckpoint(FILE_PATH_WEIGHTS_MODEL, monitor='val_get_f1', verbose=1, save_best_only=True, mode='max')
# checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

print(f"FILE_PATH_WEIGHTS_EMB_MODEL = {FILE_PATH_WEIGHTS_MODEL}")

## Bidirectional-LSTM Model.

In [31]:
emb_dim=32 # need this
emb_model_content = tf.keras.Sequential([ 
    # tf.keras.layers.Input(shape=(MAX_LEN_DOC_CONTENT,)) , 
    tf.keras.layers.Input(shape=(LEN_DOC_CONTENT,)) , 
    tf.keras.layers.Embedding(vocab_size, output_dim=emb_dim),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64,return_sequences=True,dropout=0.3),merge_mode='ave'),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32,return_sequences=True,dropout=0.2),merge_mode='ave'),
    tf.keras.layers.LSTM(32), # can't add LSTM layers one after another, need return-seq = True
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(8, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    # tf.keras.layers.Dense(4, activation='relu'),
    # tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(2, activation="softmax")
    # tf.keras.layers.Dense(1, activation="sigmoid")
])


#emb_model_content.load_weights(FILE_PATH_WEIGHTS_MODEL)
emb_model_content.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc', get_f1, get_recall])
print(emb_model_content.summary())
# Creating best-model checkpoint

checkpoint = ModelCheckpoint(FILE_PATH_WEIGHTS_MODEL, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
# checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

print(f"FILE_PATH_WEIGHTS_EMB_MODEL = {FILE_PATH_WEIGHTS_MODEL}")

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 400, 32)           7613824   
_________________________________________________________________
bidirectional_2 (Bidirection (None, 400, 128)          49664     
_________________________________________________________________
bidirectional_3 (Bidirection (None, 400, 64)           41216     
_________________________________________________________________
lstm_5 (LSTM)                (None, 32)                12416     
_________________________________________________________________
dense_3 (Dense)              (None, 16)                528       
_________________________________________________________________
dropout_2 (Dropout)          (None, 16)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 8)                

## Bidirectional-GRU Model.

In [ ]:
emb_dim=32 # need this
emb_model_content = tf.keras.Sequential([ 
    # tf.keras.layers.Input(shape=(MAX_LEN_DOC_CONTENT,)) , 
    tf.keras.layers.Input(shape=(LEN_DOC_CONTENT,)) , 
    tf.keras.layers.Embedding(vocab_size, output_dim=emb_dim),
    tf.keras.layers.Bidirectional(tf.keras.layers.GRU(64,return_sequences=True,dropout=0.3),merge_mode='ave'),
    tf.keras.layers.Bidirectional(tf.keras.layers.GRU(32,return_sequences=True,dropout=0.2),merge_mode='ave'),
    tf.keras.layers.GRU(32), # can't add LSTM layers one after another, need return-seq = True
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(8, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    # tf.keras.layers.Dense(4, activation='relu'),
    # tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(2, activation="softmax")
    # tf.keras.layers.Dense(1, activation="sigmoid")
])

#emb_model_content.load_weights(FILE_PATH_WEIGHTS_MODEL)

#emb_model_content.load_weights("/content/drive/My Drive/SAVED_MODELS/Weights/emb-content_with_synthsis-32D-22-Aug-2am-weights.best.hdf5")
emb_model_content.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc', get_f1, get_recall])
print(emb_model_content.summary())

# Creating best-model checkpoint

checkpoint = ModelCheckpoint(FILE_PATH_WEIGHTS_MODEL, monitor='val_get_f1', verbose=1, save_best_only=True, mode='max')
# checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

print(f"FILE_PATH_WEIGHTS_EMB_MODEL = {FILE_PATH_WEIGHTS_MODEL}")

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 400, 32)           7613824   
_________________________________________________________________
bidirectional (Bidirectional (None, 400, 64)           37632     
_________________________________________________________________
bidirectional_1 (Bidirection (None, 400, 32)           18816     
_________________________________________________________________
gru_2 (GRU)                  (None, 32)                6336      
_________________________________________________________________
dense (Dense)                (None, 16)                528       
_________________________________________________________________
dropout (Dropout)            (None, 16)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 8)                 1

## CNN Model.

In [ ]:
emb_dim=32 # need this
emb_model_content = tf.keras.Sequential([ 
    # tf.keras.layers.Input(shape=(MAX_LEN_DOC_CONTENT,)) , 
    tf.keras.layers.Input(shape=(LEN_DOC_CONTENT,)) , 
    tf.keras.layers.Embedding(vocab_size, output_dim=emb_dim),
    tf.keras.layers.Conv1D(filters=128, kernel_size=6, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.MaxPooling1D(pool_size=1),
    tf.keras.layers.Conv1D(filters=64, kernel_size=6, activation='relu'),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.MaxPooling1D(pool_size=1),
    tf.keras.layers.Conv1D(filters=32, kernel_size=5, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.MaxPooling1D(pool_size=1),
    tf.keras.layers.Conv1D(filters=16, kernel_size=5, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.MaxPooling1D(pool_size=1),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(8, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(2, activation='softmax')
])

#emb_model_content.load_weights(FILE_PATH_WEIGHTS_MODEL)

#emb_model_content.load_weights("/content/drive/My Drive/SAVED_MODELS/Weights/emb-content_with_synthsis-32D-22-Aug-2am-weights.best.hdf5")
emb_model_content.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc', get_f1, get_recall])
print(emb_model_content.summary())

# Creating best-model checkpoint

checkpoint = ModelCheckpoint(FILE_PATH_WEIGHTS_MODEL, monitor='val_get_f1', verbose=1, save_best_only=True, mode='max')
# checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

print(f"FILE_PATH_WEIGHTS_EMB_MODEL = {FILE_PATH_WEIGHTS_MODEL}")

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 400, 32)           7613824   
_________________________________________________________________
conv1d (Conv1D)              (None, 395, 128)          24704     
_________________________________________________________________
dropout (Dropout)            (None, 395, 128)          0         
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 395, 128)          0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 390, 64)           49216     
_________________________________________________________________
dropout_1 (Dropout)          (None, 390, 64)           0         
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 390, 64)           0

## C-LSTM Model.

In [ ]:
emb_dim=32 # need this
emb_model_content = tf.keras.Sequential([ 
    # tf.keras.layers.Input(shape=(MAX_LEN_DOC_CONTENT,)) , 
    tf.keras.layers.Input(shape=(LEN_DOC_CONTENT,)) , 
    tf.keras.layers.Embedding(vocab_size, output_dim=emb_dim),
    tf.keras.layers.Conv1D(filters=128, kernel_size=6, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.MaxPooling1D(pool_size=1),
    tf.keras.layers.Conv1D(filters=64, kernel_size=6, activation='relu'),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.MaxPooling1D(pool_size=1),
    tf.keras.layers.LSTM(64,return_sequences=True,dropout=0.2),
    tf.keras.layers.LSTM(32),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(8, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    # tf.keras.layers.Dense(4, activation='relu'),
    # tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(2, activation="softmax")
    
])

#emb_model_content.load_weights(FILE_PATH_WEIGHTS_MODEL)

#emb_model_content.load_weights("/content/drive/My Drive/SAVED_MODELS/Weights/emb-content_with_synthsis-32D-22-Aug-2am-weights.best.hdf5")
emb_model_content.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc', get_f1, get_recall])
print(emb_model_content.summary())

# Creating best-model checkpoint

checkpoint = ModelCheckpoint(FILE_PATH_WEIGHTS_MODEL, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
# checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

print(f"FILE_PATH_WEIGHTS_EMB_MODEL = {FILE_PATH_WEIGHTS_MODEL}")

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 400, 32)           7613824   
_________________________________________________________________
conv1d (Conv1D)              (None, 395, 128)          24704     
_________________________________________________________________
dropout (Dropout)            (None, 395, 128)          0         
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 395, 128)          0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 390, 64)           49216     
_________________________________________________________________
dropout_1 (Dropout)          (None, 390, 64)           0         
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 390, 64)           0

In [26]:
## Form labels, encode class values as integers
labels_whole = df_whole["label"].values
labels_as_list = list(labels_whole)
encoder = LabelEncoder()
encoder.fit(labels_as_list)
encoded_Y_WHOLE = encoder.transform(labels_as_list)
# convert integers to dummy variables (i.e. one hot encoded)
# dummy_Y_whole = np_utils+.to_categorical(encoded_Y_WHOLE)
dummy_Y_whole = to_categorical(encoded_Y_WHOLE)
print(dummy_Y_whole)
print(labels_as_list)

[[0. 1.]
 [0. 1.]
 [0. 1.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [27]:
## Form X
print(f"LEN_DOC_CONTENT = {LEN_DOC_CONTENT}")
encoded_contents = df_whole['encoded_contents'].values
padded_docs_content = pad_sequences(encoded_contents, maxlen=LEN_DOC_CONTENT, padding='pre')
print(padded_docs_content[0].shape)

LEN_DOC_CONTENT = 400
(400,)


In [28]:
(classes, counts) = np.unique(df_whole["label"], return_counts=True)
counts_normalized = counts/np.sum(counts)
# print(counts_normalized)
normalized_weight_of_0, normalized_weight_of_1 = counts_normalized
print(counts, counts_normalized)

### inverse the weights to have balance
dict_class_weight = {
    0: normalized_weight_of_1,
    1: normalized_weight_of_0    
}

print(dict_class_weight)

[ 1484 43803] [0.03276879 0.96723121]
{0: 0.9672312142557467, 1: 0.03276878574425332}


In [29]:
## https://machinelearningmastery.com/save-load-keras-deep-learning-models/
EPOCHS = 30
BATCH_SIZE = 32 # use 128 for faster training
VAL_SPLIT = 0.1

print(f"FILE_MODEL = {FILE_MODEL}")

emb_model_content.fit(padded_docs_content, dummy_Y_whole,
              epochs = EPOCHS, batch_size = BATCH_SIZE, verbose = 1, validation_split = VAL_SPLIT,
              callbacks = callbacks_list, class_weight = dict_class_weight)

#emb_model_content.save(FILE_MODEL) ## using tensorflow.keras's api


FILE_MODEL = /content/drive/My Drive/SAVED_MODELS/classifier-bi-LSTM2-22-Aug-2am.h5
Epoch 1/30
1274/1274 [==============================] - ETA: 0s - loss: 0.0263 - acc: 0.8018 - get_f1: 0.8019 - get_recall: 0.8019
Epoch 00001: val_acc improved from -inf to 0.94922, saving model to /content/drive/My Drive/SAVED_MODELS/Weights/classifier-bi-LSTM2-22-Aug-2am-weights.best.hdf5
1274/1274 [==============================] - 174s 137ms/step - loss: 0.0263 - acc: 0.8018 - get_f1: 0.8019 - get_recall: 0.8019 - val_loss: 0.3235 - val_acc: 0.9492 - val_get_f1: 0.9494 - val_get_recall: 0.9494
Epoch 2/30
1274/1274 [==============================] - ETA: 0s - loss: 0.0105 - acc: 0.9701 - get_f1: 0.9701 - get_recall: 0.9701
Epoch 00002: val_acc improved from 0.94922 to 0.96798, saving model to /content/drive/My Drive/SAVED_MODELS/Weights/classifier-bi-LSTM2-22-Aug-2am-weights.best.hdf5
1274/1274 [==============================] - 174s 137ms/step - loss: 0.0105 - acc: 0.9701 - get_f1: 0.9701 - get_rec

In [32]:
emb_model_content.save(FILE_MODEL) ## using tensorflow.keras's api

In [ ]:
# score = loaded_model.evaluate(X, Y, verbose=0)
# print("%s: %.2f%%" % (loaded_model.metrics_names[1], score[1]*100))

In [ ]:
## https://stackoverflow.com/questions/51700351/valueerror-unknown-metric-function-when-using-custom-metric-in-keras

dependencies = {
    # 'auc_roc': auc_roc
    'get_recall': get_recall,
    'get_f1': get_f1
}

# model = keras.models.load_model(self.output_directory + 'best_model.hdf5', custom_objects=dependencies)
print(f"FILE_MODEL = {FILE_MODEL}")
emb_model_content = tf.keras.models.load_model(FILE_MODEL, custom_objects=dependencies)
print(emb_model_content.summary())

FILE_MODEL = /content/drive/My Drive/SAVED_MODELS/emb-content-32D-22-Aug-2am.h5
Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 400, 32)           7613824   
_________________________________________________________________
bidirectional_4 (Bidirection (None, 400, 64)           49664     
_________________________________________________________________
bidirectional_5 (Bidirection (None, 400, 32)           24832     
_________________________________________________________________
lstm_8 (LSTM)                (None, 32)                8320      
_________________________________________________________________
dense_6 (Dense)              (None, 16)                528       
_________________________________________________________________
dropout_4 (Dropout)          (None, 16)                0         
________________________________________

In [ ]:
## https://stackoverflow.com/questions/51700351/valueerror-unknown-metric-function-when-using-custom-metric-in-keras

dependencies = {
    # 'auc_roc': auc_roc
    'get_recall': get_recall,
    'get_f1': get_f1
}

# model = keras.models.load_model(self.output_directory + 'best_model.hdf5', custom_objects=dependencies)
print(f"FILE_MODEL = {FILE_MODEL}")
emb_model_content = tf.keras.models.load_model(FILE_MODEL, custom_objects=dependencies)
print(emb_model_content.summary())